# Slide puzzle (corridor) (****)

Slide coins to empty positions. The red coin should end up at the empty spot at the right
end. Model this with SMT and let Z3 find a solution.

![./slide_puzzle_corridor.png](./slide_puzzle_corridor.png)

## Approach

I did not want to use my code from previous semester since A) I lost my train of thought about that approach since I did not document it very well, and B) after some re-evaluation of the code I could not forsee whether the old approach would even work or not, so I decided to start from scratch.

Each time I wanted to test something, or could not get something to work, I would make a copy of my file and increment a counter by 1 in the filename, so that I could keep track of my progress this time. Each file is a step in the process of getting to the final solution. I will explain each file in the order they were created.

---

### Premise

The entire premise of my new approach was to make the following work:
```smt
(declare-const Coin_A Int)
(declare-const Coin_B Int)
; ...

; The first argument is the coin identifier, the second is the point in time
(declare-fun LocX (Int Int) Int)
(declare-fun LocY (Int Int) Int)
; ...

(assert
(and
    ; Constrains go here

    (exists ((T Int))
        (and
            (= (LocX Coin_A T) 9)
            (= (LocY Coin_A T) 0)
        )
    )

) ; End of AND
) ; End of ASSERT
```

---

### Progress towards solution

> `slide_puzzle_corridor.smt`

The new approach started with modeling the coins and squares, but after some thought I figured out you can model just the coins in terms of X/Y locations and constrain those, essentially mitigating the need to also model the squares in code (as seperate objects).

> `slide_puzzle_corridor_2.smt` and `slide_puzzle_corridor_3.smt`

I then decided to work on making the coins move but had a hard time gettting this to work. I eventually got this to work by creating a new smt file and stripping the issue down to its core: making a abstract function return increasing values, essentially making a single coin move to the right (possitive X)

> `slide_puzzle_corridor_4.smt`

I then reintroduced all the other stuff from the previous solution file (puzzle boundaries, extra coins, etc...) and started working on making the movement of the coin towards possitive X work given more constrains.

However, it seemed that the LocX / LocY approach was very messy (`get-model` looked very cluttered). I then spoke to Herman about visualizing the model and he gave me 2 points of feedback:
- make the puzzle smaller (less coins and less squares)
- use custom data types

> `slide_puzzle_corridor_5.smt`


## Visuallizing the output

In [161]:
# ...

## Code generation

In [171]:
def generate_puzzle(
    name: str,
    adjecency: dict,
    starting_positions: dict,
):
    # The puzzle definition will be contained within a function
    adjecency_function = """
(define-fun {0} () Bool
    (and
{1}
{2}
        BidirecitonalAdjecency
        
{3}
    )
)
"""

    # Use the adjecency dictionary to set truthfulness of Adjacenct function
    adjecency_NODE = "\t\t(= (Adjacenct {0} {1}) true)"
    adjecency_PREDEFINED = ["\t\t; Predefined adjecency"]
    for source, v in adjecency.items():
        for target in v:
            adjecency_PREDEFINED.append(adjecency_NODE.format(source, target))

    adjecency_NODES = "\n".join(adjecency_PREDEFINED)

    adjecency_DEFAULT_FALSE = """
        ; Adjecency is always false, unless explicitly specified above
        (forall ((S1 Square) (S2 Square))
            (implies
                (not
                    (or{0}
                    )
                )
                ; Default to false
                (= (Adjacenct S1 S2) false)
            )
        )
"""
    adjecency_ORITEM = """
                        (and (= S1 {0}) (= S2 {1}))
                        (and (= S1 {1}) (= S2 {0}))"""
    adjecency_ORITEMS = []

    for source, v in adjecency.items():
        for target in v:
            adjecency_ORITEMS.append(adjecency_ORITEM.format(source, target))

    adjecency_ORITEMS = "\n".join(adjecency_ORITEMS)
    adjecency_DEFAULT_FALSE = adjecency_DEFAULT_FALSE.format(adjecency_ORITEMS)

    starting_positions_ANDITEM = "\t\t(= (Pos {0} 0) {1})"
    starting_positions_ANDITEMS = ["\t\t; Starting positions"]

    for coin, pos in starting_positions.items():
        starting_positions_ANDITEMS.append(starting_positions_ANDITEM.format(coin, pos))

    starting_positions_ANDITEMS = "\n".join(starting_positions_ANDITEMS)

    return adjecency_function.format(
        name,
        adjecency_NODES,
        adjecency_DEFAULT_FALSE,
        starting_positions_ANDITEMS
    ).replace("\t", "    ")

In [172]:
puzzle_name = "Puzzle1"
adjecency = {
    "Start": [ "One" ],
    "One": [ "Two" ],
    "Two": [ "Three", "Five" ],
    "Three": [ "Four" ],
    "Four": [ "End", "Six" ],
}
starting_positions = {
    "Red":  "Start",
    "A":    "One",
    "B":    "Two",
    "C":    "Three",
    "D":    "Four",
}

print(
    generate_puzzle(
        puzzle_name,
        adjecency,
        starting_positions,
))


(define-fun Puzzle1 () Bool
    (and
        ; Predefined adjecency
        (= (Adjacenct Start One) true)
        (= (Adjacenct One Two) true)
        (= (Adjacenct Two Three) true)
        (= (Adjacenct Two Five) true)
        (= (Adjacenct Three Four) true)
        (= (Adjacenct Four End) true)
        (= (Adjacenct Four Six) true)

        ; Adjecency is always false, unless explicitly specified above
        (forall ((S1 Square) (S2 Square))
            (implies
                (not
                    (or
                        (and (= S1 Start) (= S2 One))
                        (and (= S1 One) (= S2 Start))

                        (and (= S1 One) (= S2 Two))
                        (and (= S1 Two) (= S2 One))

                        (and (= S1 Two) (= S2 Three))
                        (and (= S1 Three) (= S2 Two))

                        (and (= S1 Two) (= S2 Five))
                        (and (= S1 Five) (= S2 Two))

                        (and (= S1 Three) (= S2 Four

In [167]:
MaxT = 20
getvalue = """
(get-value (
{0}
))
"""
posred = "(Pos Red {0})"
redpositions = []

for T in range(0, MaxT + 2):
    redpositions.append(f"\t{posred.format(T)}\n")

print(getvalue.format("".join(redpositions)))


(get-value (
	(Pos Red 0)
	(Pos Red 1)
	(Pos Red 2)
	(Pos Red 3)
	(Pos Red 4)
	(Pos Red 5)
	(Pos Red 6)
	(Pos Red 7)
	(Pos Red 8)
	(Pos Red 9)
	(Pos Red 10)
	(Pos Red 11)
	(Pos Red 12)
	(Pos Red 13)
	(Pos Red 14)
	(Pos Red 15)
	(Pos Red 16)
	(Pos Red 17)
	(Pos Red 18)
	(Pos Red 19)
	(Pos Red 20)
	(Pos Red 21)

))

